# **How to Create Interactive Covid-19 Map in Python Using Pandas and Plotly**

As Covid-19 is still a critical world issue, logistic distribution, transportation safety, education and various other sector is disrupted. To support decision making that consider Covid-19, data representing cases of covid and its location is beneficial. A powerful way of visualizing the aforementioned data is by utilizing map. In this case, the code below provide steps to create static and interactive daily time-series map of confirmed cases, recovered cases, and death tolls of Covid-19.

***Additional Notes***

Critics and suggestions is very appreciated.

# Import Required Libraries
Libraries used in this codes are:
* Pandas to manipulate create and manipulate dataframe
* Plotly to visualize dataframa

In [ ]:
import pandas as pd 
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from shapely.geometry import Point, LineString, Polygon
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Retreiving Global Daily Covid-19 Data
Raw data on daily confirmed cases, recovered cases and death toll for all countries is obtained from [COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19).

In [ ]:
fp_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
fp_recovered = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
fp_death = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'

Read raw data using pandas

In [ ]:
# Read data
skip_col = 'Province/State'
confirmed = pd.read_csv(fp_confirmed, usecols=lambda x: x not in skip_col)
recovered = pd.read_csv(fp_recovered, usecols=lambda x: x not in skip_col)
death = pd.read_csv(fp_death, usecols=lambda x: x not in skip_col)

Rename column and create column containing the total confirmed cases, recovered cases and death toll for all countries.

In [ ]:
df_list = [confirmed, recovered, death]

# Rename columns
def rename_cols(x):
    for i in x:
        i = i.rename(columns={'Country/Region':'Country'}, inplace=True)

# Total confirmed/recovered/death case
def total_case(x):
    for i in x:
        i['total'] = i.iloc[:,-1]

In [ ]:
# Rename column
rename_cols(df_list)

In [ ]:
total_case(df_list)

In [ ]:
# Group by countries
conf_countries = confirmed.groupby('Country').sum().reset_index()
reco_countries = recovered.groupby('Country').sum().reset_index()
death_countries = death.groupby('Country').sum().reset_index()

In [ ]:
# Delete duplicate
list_case = [conf_countries, reco_countries, death_countries]
for i in list_case:
    i = i.drop_duplicates(subset=['Country'])

# Create Static Map Visualizing Total Confirmed Case
The code below shows steps to create map of total confirmed case for all countries. Codes can be modified to show total recovered cases or total death toll for all countries.

In [ ]:
# Create Confirmed Case Choropleth
fig = go.Figure(data=go.Choropleth(
    locations = confirmed['Country'],
    locationmode = 'country names',
    z = confirmed.iloc[:,-1],
    colorscale = 'Reds',
    marker_line_color = 'black',
    marker_line_width = 0.5,
))
fig.update_layout(
    title_text = 'Total Confirmed Cases',
    title_x = 0.5,
    margin=dict(l=30, r=30, t=30, b=30),
    geo=dict(
        showframe = False,
        showcoastlines = False,
        projection_type = 'equirectangular'
    )
)

# Create Interactive Map
The code below shows steps to create interactive daily time series of confirmed cases, recovered cases, and death toll for all countries.

In [ ]:
# Transpose part of data to create time series animation
confirmed_melt = conf_countries.melt(['Country', 'Lat', 'Long'], var_name='Date', value_name='Confirmed')
recovered_melt = reco_countries.melt(['Country', 'Lat', 'Long'], var_name='Date', value_name='Recovered')
death_melt = death_countries.melt(['Country', 'Lat', 'Long'], var_name='Date', value_name='Death')

# Remove no-data entries
confirmed_melt = confirmed_melt.loc[confirmed_melt['Confirmed'] > 0]
recovered_melt = recovered_melt.loc[recovered_melt['Recovered'] > 0]
death_melt = death_melt.loc[death_melt['Death'] > 0]

In [ ]:
# Create Interactive Map
def case_map(x):
    fig = px.choropleth(x, 
                    locations="Country", 
                    locationmode = "country names",
                    color=x.columns[-1], 
                    hover_name="Country", 
                    animation_frame="Date"
                   )
    fig.update_layout(
    title_text = 'Global '+x.columns[-1]+' Case of Corovavirus',
    title_x = 0.5,
    margin=dict(l=10, r=10, t=100, b=100),
    width = 750,
    height = 500,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    fig.show()

In [ ]:
case_map(confirmed_melt)

In [ ]:
case_map(recovered_melt)

In [ ]:
case_map(death_melt)